In [4]:
filename = "Canteen.mp4"
frame_width, frame_height = 1280,720
# modelPath = "./yolo-head-detection.pt"
modelPath = "./yolov8s.pt"


In [9]:
import torch

print("Torch version:",torch.__version__)

print("Is CUDA enabled?",torch.cuda.is_available())

Torch version: 2.2.0+cpu
Is CUDA enabled? False


## BOTSORT

In [6]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from boxmot import BoTSORT
from pathlib import Path
frame_width, frame_height = 1280,720
model = YOLO(modelPath)

def calculate_color_based_on_id(id):
  colors = [[255, 0, 0], # Red
          [0, 255, 0], # Green|
          [0, 0, 255], # Blue
          [255, 255, 0], # Yellow
          [255, 0, 255], # Magenta
          [0, 255, 255], # Cyan
          [128, 128, 128], # Gray
          [255, 255, 255], # White
          [0, 0, 0], # Black
          [192, 192, 192]]
  return colors[id % len(colors) ]



cap = cv2.VideoCapture(f"./input_videos/{filename}")
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
ret, frame = cap.read() 
tracker = BoTSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False,track_buffer=60*15,frame_rate=60)
thickness = 2
color = (0,0,255)
# cap_out = cv2.VideoWriter(f"output_videos/botsort_{filename}", cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
#                           (frame.shape[1], frame.shape[0]))
count = 0
id_set = set()
while(cap.isOpened()):
    count +=1 
    detections = model.predict(frame,verbose=False)[0]
    results = []
    if not ret:
        break

    if np.array(detections.boxes.data.tolist()).ndim < 2:
        results = [[0, 0, 0, 0, 0.0922948837280273, 0]] 
    try:
        ts = tracker.update(np.array(detections.boxes.data.tolist()), frame)
        xyxys = ts[:,0:4].astype('int') # float64 to int
        ids = ts[:, 4].astype('int') # float64 to int 
        confs = ts[:, 5]
        clss = ts[:, 6]
        if ts.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):

                if cls != 0:
                    continue
                id_set.add(id)
                cv2.rectangle(
                    frame,
                    (xyxy[0], xyxy[1]),
                    (xyxy[2], xyxy[3]),
                    calculate_color_based_on_id(id),
                    thickness
                )
                cv2.putText(
                    frame,
                    f'id: {id}',
                    (xyxy[0], xyxy[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    calculate_color_based_on_id(id),
                    thickness
                )
            cv2.putText(
                    frame,
                    f'Count: {len(id_set)}',
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            cv2.putText(
                    frame,
                    f'Frame: {count}',
                    (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            
            cv2.imshow('Frame', frame)

            # cap_out.write(frame)

    except Exception as e:
         print("Error:",e)
         pass
    ret, frame = cap.read()         
    if cv2.waitKey(25) & 0xFF == ord('q'): 
            break

cap.release() 
cv2.destroyAllWindows() 
print(f"botsort_{filename}")

2024-02-10 16:04:10.609 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2024-02-10 16:04:10.611 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from boxmot import BoTSORT
from pathlib import Path
frame_width, frame_height = 1280,720
model = YOLO("yolo-head-detection.pt")

def calculate_color_based_on_id(id):
  colors = [[255, 0, 0], # Red
          [0, 255, 0], # Green|
          [0, 0, 255], # Blue
          [255, 255, 0], # Yellow
          [255, 0, 255], # Magenta
          [0, 255, 255], # Cyan
          [128, 128, 128], # Gray
          [255, 255, 255], # White
          [0, 0, 0], # Black
          [192, 192, 192]]
  return colors[id % len(colors) ]



cap = cv2.VideoCapture(f"./input_videos/{filename}")
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
ret, frame = cap.read() 
tracker = BoTSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False)
thickness = 2
color = (0,0,255)
# cap_out = cv2.VideoWriter(f"output_videos/botsort_{filename}", cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
#                           (frame.shape[1], frame.shape[0]))

while(cap.isOpened()):
    id_set = set()
    detections = model.predict(frame,verbose=False)[0]
    results = []
    if not ret:
        break

    if np.array(detections.boxes.data.tolist()).ndim < 2:
        results = [[0, 0, 0, 0, 0.0922948837280273, 0]] 
    try:
        ts = tracker.update(np.array(detections.boxes.data.tolist()), frame)
        xyxys = ts[:,0:4].astype('int') # float64 to int
        ids = ts[:, 4].astype('int') # float64 to int 
        confs = ts[:, 5]
        clss = ts[:, 6]
        if ts.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):

                if cls != 0:
                    continue
                id_set.add(id)
                cv2.rectangle(
                    frame,
                    (xyxy[0], xyxy[1]),
                    (xyxy[2], xyxy[3]),
                    calculate_color_based_on_id(id),
                    thickness
                )
                cv2.putText(
                    frame,
                    f'id: {id}',
                    (xyxy[0], xyxy[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    calculate_color_based_on_id(id),
                    thickness
                )
            print(len(id_set))
            cv2.putText(
                    frame,
                    f'Count: {len(id_set)}',
                    (10, 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            cv2.imshow('Frame', frame)
            # cap_out.write(frame)

    except Exception as e:
         print("Error:",e)
         pass
    ret, frame = cap.read()         
    if cv2.waitKey(25) & 0xFF == ord('q'): 
            break

cap.release() 
cv2.destroyAllWindows() 
print(f"botsort_{filename}")

2023-11-04 04:25:41.993 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2023-11-04 04:25:41.994 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


12
12
12
12
15
15
16
16
16
16
15
15
15
15
15
14
14
14
15
15
15
15
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
13
13
13
14
13
13
15
15
14
14
13
13
13
13
12
12
12
12
12
11
11
11
11
10
11
12
12
12
12
12
12
11
11
11
10
10
10
10
10
10
10
11
11
11
11
11
11
11
11
11
11
11
11
11
11
12
12
12
12
12
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
14
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
13
13
13
13
13
12
12
12
12
12
13
14
14
14
14
14
14
14
14
14
14
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
14
14
14
14
14
14
1

WARNING ⚠️ 'source' is missing. Using 'source=/home/loki/Development/major-project/RealTime-Webcam/venv/lib/python3.11/site-packages/ultralytics/assets'.


13
botsort_Canteen.mp4


## StrongSort

In [15]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from boxmot import BoTSORT,StrongSORT
from pathlib import Path
model = YOLO(modelPath)

def calculate_color_based_on_id(id):
  colors = [[255, 0, 0], # Red
          [0, 255, 0], # Green
          [0, 0, 255], # Blue
          [255, 255, 0], # Yellow
          [255, 0, 255], # Magenta
          [0, 255, 255], # Cyan
          [128, 128, 128], # Gray
          [255, 255, 255], # White
          [0, 0, 0], # Black
          [192, 192, 192]]
  return colors[id % len(colors) ]



cap = cv2.VideoCapture(f"./input_videos/{filename}")
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
ret, frame = cap.read() 
tracker = StrongSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False)
thickness = 2
color = (0,0,255)
while(cap.isOpened()):
    id_set = set()
    detections = model.predict(frame,verbose=False)[0]
    results = []
    if not ret:
        break

    if np.array(detections.boxes.data.tolist()).ndim < 2:
        results = [[0, 0, 0, 0, 0.0922948837280273, 0]] 
    try:
        ts = tracker.update(np.array(detections.boxes.data.tolist()), frame)
        xyxys = ts[:,0:4].astype('int') # float64 to int
        ids = ts[:, 4].astype('int') # float64 to int 
        confs = ts[:, 5]
        clss = ts[:, 6]
        if ts.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):

                if cls != 0:
                    continue
                id_set.add(id)
                cv2.rectangle(
                    frame,
                    (xyxy[0], xyxy[1]),
                    (xyxy[2], xyxy[3]),
                    calculate_color_based_on_id(id),
                    thickness
                )
                cv2.putText(
                    frame,
                    f'id: {id}',
                    (xyxy[0], xyxy[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    calculate_color_based_on_id(id),
                    thickness
                )
            cv2.putText(
                    frame,
                    f'Count: {len(id_set)}',
                    (10, 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            cv2.imshow('Frame', frame)
    except Exception as e:
         print("Error:",e)
         pass
    ret, frame = cap.read()         
    if cv2.waitKey(25) & 0xFF == ord('q'): 
            break

cap.release() 
cv2.destroyAllWindows() 

2023-11-04 04:52:48.231 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2023-11-04 04:52:48.232 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


Error: too many indices for array: array is 1-dimensional, but 2 were indexed


WARNING ⚠️ 'source' is missing. Using 'source=/home/loki/Development/major-project/RealTime-Webcam/venv/lib/python3.11/site-packages/ultralytics/assets'.


## ByteTrack

In [4]:
import cv2
import argparse

from ultralytics import YOLO
import supervision as sv


cap = cv2.VideoCapture(f"./input_videos/{filename}")
# cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)


# model = YOLO('yolo-head-detection.pt')
model = YOLO('yolov8l.pt')

CLASS_NAMES_DICT = model.model.names
box_annotator = sv.BoxAnnotator()
# mask_annotator = sv.MaskAnnotator()
tracker = sv.ByteTrack(frame_rate=30,track_buffer=120)
print(model.model.names)

ret,frame = cap.read()
cap_out = cv2.VideoWriter(f"out.mp4", cv2.VideoWriter_fourcc(*'MP4V'), cap.get(cv2.CAP_PROP_FPS),
                      (frame.shape[1], frame.shape[0]))
while ret:
    ids = set()
    result = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)  
    detections = detections[(detections.class_id==0)]
    detections = tracker.update_with_detections(detections=detections)
    labels  = []
    for _, _, confidence, class_id, tracker_id in detections:
        labels.append(f"#{tracker_id} {model.model.names[class_id]} {confidence:0.2f}")
        ids.add(tracker_id)
    
    annotated_image = box_annotator.annotate(frame, detections=detections,labels=labels)
    # annotated_image = mask_annotator.annotate(scene=frame,detections=detections,opacity=0.5)
    cv2.putText(
                    annotated_image,
                    f"Count {len(ids)}",
                    (10, 20),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    2
                )
    cv2.imshow("YoloV8",annotated_image)
    cap_out.write(annotated_image)
    # ESC -> 27
    if cv2.waitKey(30) == 27:
        break
    ret,frame = cap.read()


{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
QObject::moveToThread: Current thread (0x55b351395560) is not the object's thread (0x55b350f145c0).
Cannot move to target thread (0x55b351395560)

qt.qpa.plugin: Could not load the Qt platform plugin "wayland" in "/home/loki/Development/major-project/RealTime-Webcam/venv/lib/python3.11/site-packages/cv2/qt/plugins" even though it was found.


: 

# Deep SORT

In [6]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
from boxmot import BoTSORT,StrongSORT,DeepOCSORT
from pathlib import Path
model = YOLO("./yolo-head-detection.pt")

def calculate_color_based_on_id(id):
  colors = [[255, 0, 0], # Red
          [0, 255, 0], # Green
          [0, 0, 255], # Blue
          [255, 255, 0], # Yellow
          [255, 0, 255], # Magenta
          [0, 255, 255], # Cyan
          [128, 128, 128], # Gray
          [255, 255, 255], # White
          [0, 0, 0], # Black
          [192, 192, 192]]
  return colors[id % len(colors) ]



cap = cv2.VideoCapture(f"./input_videos/{filename}")
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_width)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,frame_height)
ret, frame = cap.read() 
tracker = DeepOCSORT(model_weights=Path('osnet_x0_25_msmt17.pt'),device="cuda:0",fp16=False)
thickness = 2
color = (0,0,255)
while(cap.isOpened()):
    id_set = set()
    detections = model.predict(frame,verbose=False)[0]
    results = []
    if not ret:
        break

    if np.array(detections.boxes.data.tolist()).ndim < 2:
        results = [[0, 0, 0, 0, 0.0922948837280273, 0]] 
    try:
        ts = tracker.update(np.array(detections.boxes.data.tolist()), frame)
        xyxys = ts[:,0:4].astype('int') # float64 to int
        ids = ts[:, 4].astype('int') # float64 to int 
        confs = ts[:, 5]
        clss = ts[:, 6]
        if ts.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):

                if cls != 0:
                    continue
                id_set.add(id)
                cv2.rectangle(
                    frame,
                    (xyxy[0], xyxy[1]),
                    (xyxy[2], xyxy[3]),
                    calculate_color_based_on_id(id),
                    thickness
                )
                cv2.putText(
                    frame,
                    f'id: {id}',
                    (xyxy[0], xyxy[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    calculate_color_based_on_id(id),
                    thickness
                )
            cv2.putText(
                    frame,
                    f'Count: {len(id_set)}',
                    (10, 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0,0,0),
                    thickness
                )
            cv2.imshow('Frame', frame)
    except Exception as e:
         print("Error:",e)
         pass
    ret, frame = cap.read()         
    if cv2.waitKey(25) & 0xFF == ord('q'): 
            break

cap.release() 
cv2.destroyAllWindows() 

2023-11-04 04:41:13.020 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2023-11-04 04:41:13.021 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')
